In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip3 install clearml

In [ ]:
!pip install ultralytics

In [ ]:
!pip install roboflow

In [ ]:
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
import cv2
from roboflow import Roboflow

In [ ]:
import clearml

In [ ]:
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=ZX6KFUCWS3CBW3DGZGZF
%env CLEARML_API_SECRET_KEY=2zAK6RRUMfNviS7L7jbdRbU5IQRYNVjqrdWaLpOJDuMPvK0f8A

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=ZX6KFUCWS3CBW3DGZGZF
env: CLEARML_API_SECRET_KEY=2zAK6RRUMfNviS7L7jbdRbU5IQRYNVjqrdWaLpOJDuMPvK0f8A


In [ ]:
from clearml import Task
task = Task.init(project_name="Yolov8 finetune", task_name="train cars")

In [ ]:
def cut_frames(video:str, compress:float) -> None:
  """
  :video: path to the source video.
  :compress: percentage value of compression. between 0 and 1.
  """
  cap = cv2.VideoCapture(video)
  droprate = (1 - compress) * 10
  if not cap.isOpened():
      print("Error: Could not open the input video file.")
      exit()


  output_file = 'output_video.mp4'
  fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Change this to your desired codec
  frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
  frame_rate = int(cap.get(cv2.CAP_PROP_FPS))

  out = cv2.VideoWriter(output_file, fourcc, frame_rate, frame_size, isColor=True)

  if not out.isOpened():
      print("Error: Could not create the output video file.")
      cap.release()
      exit()

  counter = 0
  while cap.isOpened():
      ret, frame = cap.read()

      if not ret:
          break

      if counter % droprate != 0:
        out.write(frame)
      counter+=1

  cap.release()
  out.release()

In [ ]:
rf = Roboflow(api_key="MB18qO6D3oKJK8Chrz1f")
project = rf.workspace("mohamed-traore-2ekkp").project("face-detection-mik1i")
version = project.version(24)
face_dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.46, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Face-Detection-24 in yolov8:: 100%|██████████| 7643/7643 [00:01<00:00, 7426.44it/s]


In [ ]:
project_vehicle = rf.workspace("roboflow-100").project("vehicles-q0x2v")
version_vehicle = project_vehicle.version(2)
vehicle_dataset = version_vehicle.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.46, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to vehicles-2 in yolov8:: 100%|██████████| 8128/8128 [00:02<00:00, 2886.65it/s]


In [ ]:
# Load a model
model = YOLO('yolov8n.yaml')  # build a new model from YAML
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)
model = YOLO('yolov8n.yaml').load('yolov8n.pt')  # build from YAML and transfer weights

100%|██████████| 6.23M/6.23M [00:00<00:00, 81.0MB/s]


Transferred 355/355 items from pretrained weights


In [ ]:
for k, v in model.named_parameters():
    print(k)

In [ ]:
# Train the model
results = model.train(data="/content/Face-Detection-24/data.yaml", epochs=20, imgsz=640, plots=True, freeze=18)

In [ ]:
# Train the model
results_2 = model.train(data="/content/vehicles-2/data.yaml", epochs=10, imgsz=640, plots=True, freeze=18)

In [ ]:
!mv /content/runs/detect/train62 /content/drive/MyDrive/capstone_data/capstone2024/yolov8